In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
Year = 1   # Do not change!

CElevts = 2
LElevts = 2
ETerminals = 2
IChina = 2
Alpha = 0.9    # Inventory deterioration rate

Dom_P =  200   # Domestic Soybean price
Beta1 = Beta2 = Beta3 = 1  # Regression coefficients
Glo_P =  400   # Global Soybean price
Gamma1 = Gamma2 =Gamma3 = 1 # Regression coefficients

I_last = np.ones(CElevts)*100 # last year inventory for each country elevator

In [3]:
# Parameters
    ## from CElevts to LElevts by Trucks  e ,j
Cost_C_L = np.ones([CElevts, LElevts]).reshape(CElevts,LElevts)
    ## from LElevts to ETerminals by Barges j,k
Cost_L_E = np.ones([LElevts, ETerminals]).reshape(LElevts, ETerminals)
    ## from ETerminals to IChina by Ocean shipment k,m
Cost_E_I = np.ones([ETerminals,IChina]).reshape(ETerminals,IChina)
    ## from CElevts to Domestic Processing Facility C^P
Cost_E_F = np.ones(CElevts).reshape(CElevts)
    ##E Country elevators hoding cost C^H
Cost_E_H = np.ones([CElevts]).reshape([CElevts])
    ## Supply of each Country elevator
S = np.ones([CElevts]).reshape([CElevts])
    ## Demand of China each ports
D = np.ones([IChina]).reshape([IChina])

In [4]:
# Model
model = Model("Soybean May 28 2021 V11")

Using license file C:\Users\heish\gurobi.lic
Academic license - for non-commercial use only


In [5]:
# Vars
x = model.addVars(CElevts, LElevts, lb=0, name="x_ejt")
I = model.addVars(CElevts, lb=0, name='I_et')
f = model.addVars(CElevts, lb=0, name='f_et')
y = model.addVars(LElevts, ETerminals, lb=0, name='y_jkt')
z = model.addVars(ETerminals, IChina, lb=0, name='z_kmt')
## Slack Vars
L = model.addVar(Year, vtype=GRB.BINARY, name='L-Climate')
F = model.addVar(Year, vtype=GRB.BINARY, name='F-GOVPolicy')

In [6]:
# add constraints 2
for e in range(CElevts):
    model.addConstr(Alpha*I_last[e] + S[e] == f[e] + I[e] + quicksum(x[e,j] for j in range(LElevts)))

C:\Users\heish\anaconda3\envs\gurobi\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# add Constraints 3
for j in range(LElevts):
    model.addConstr(quicksum(x[e,j] for e in range(CElevts)) == quicksum(y[j,k] for k in range(ETerminals)))

In [8]:
# add Constratints 4
for k in range(ETerminals):
    model.addConstr(quicksum(y[j,k] for j in range(LElevts)) == quicksum(z[k,m] for m in range(IChina)))

In [9]:
# add Constratints 5
for m in range(IChina):
    model.addConstr(quicksum(z[k,m] for k in range(ETerminals)) <= D[m])

In [10]:
# add Constratints 6
model.addConstr(Dom_P == Beta1 + Beta2*L + Beta3*quicksum(I))

<gurobi.Constr *Awaiting Model Update*>

In [11]:
# add Constratints 7
model.addConstr(Glo_P == Gamma1 + Gamma2*F + Gamma3*quicksum(z))

<gurobi.Constr *Awaiting Model Update*>

In [12]:
# Obj
obj = LinExpr()
obj += quicksum((Glo_P-Cost_E_I[k,m])*z[k,m] for k in range(ETerminals) 
                                             for m in range(IChina))
obj -= quicksum(Cost_C_L[e,j]*x[e,j] for e in range(CElevts)
                                     for j in range(LElevts))
obj += quicksum((Dom_P-Cost_E_F[e])*f[e] for e in range(CElevts))
obj -= quicksum(Cost_E_H[e]*I[e] for e in range(CElevts))
obj -= quicksum(Cost_L_E[j,k]*y[j,k] for j in range(LElevts)
                                     for k in range(ETerminals))
model.setObjective(
    obj,
    GRB.MAXIMIZE
)

In [13]:
model.update()
model.optimize()
model.write('model.lp')

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 10 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x25e5125c
Variable types: 16 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
